# Isogrid Module Design for the LV4 Sub-Orbital Rocket
## Motivation
PSAS has used carbon fiber composites for generations of rockets. The introduction of isogrid aluminum pressure vessels has granted the impetus to examine isogrid's application as a general structural component. The goal is to have a design that is manufacturable by the amatuer group, but has a mass close to composite designs.

## Design Choices
### Material
For simplicity, the tank is built from a tube or pipe, eliminating the need to weld the length of the tank. The downside is that this limits the material options to what can be obtained off the shelf. A quick survey of choices found the following options, all made from 6061-T6. Note, material prices as of 2020/10

| ID (in) | Wall (in) | OD (in) | Type | Price/Ft | Source |
|---------|-----------|---------|------|----------|--------|
| 10.02 | 0.365 | 10.75 | Pipe | 270.01 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/10-nom-schedule-40-aluminum-pipe-6061-t6-extruded/pid/10619)|
| 10.02 | 0.365 | 10.75 | Pipe | 64.33 | [Midwest Steel & Al*](https://www.midweststeelsupply.com/store/6061aluminumpipe)|
| 10.50 | 0.750 | 12.00 | Tube | 826.71 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/12-od-x-0-75-wall-x-10-5-id-aluminum-round-tube-6061-t6-extruded-aluminum-association-in-/pid/21986)|
| 11.00 | 0.500 | 12.00 | Tube | 678.12 | [Metals Depot](https://www.metalsdepot.com/search?q=T3R12500)
| 11.50 | 0.250 | 12.00 | Tube | 181.63 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/12-od-x-0-25-wall-x-11-5-id-aluminum-round-tube-6061-t6-extruded/pid/23929)|
| 12.00 | 0.375 | 12.75 | Pipe | 85.10 | [Midwest Steel & Al*](https://www.midweststeelsupply.com/store/6061aluminumpipe)|

*For Midwest Steel and Al, material is considered raw and can have light scratches

Other sizes appear to be available from [Vita Needle](https://www.vitaneedle.com/aluminum-tube/) but quotes have to be requested.

6061-T6 is a an acceptable choice as it has reasonable strength, weldability and LOX compatibility. The 11.5 in inner diameter (ID) pipe was selected as it's closest to the initial LV4 multi-disciplinary design targets and least expensive option.

For initial estimates, a total tank length of 48" is assumed. This fits with available manufacuring possibilities.

In [1]:
# Stock properties
grid_id = 11.5     # in
grid_wall_t = 0.21  # in
grid_od = grid_id+2*grid_wall_t # in
grid_r = grid_od/2   # in
grid_length = 21.24144  # in

# Material properties for Al 6061-T6
E_Si = 69.0 # Young's Modules, GPa
density = 0.0975 #lb/in^3
E = 10e6 # psi
S_y = 40e3 # Yield Tensile Strength - psi
S_u = 45e3 # Ultimate Tensile Strength - psi

nu = 0.33 # Poisson's Ratio


### Safety Factor
Minimum safety factor for Base 11 is 1.5 (see requirement 3.2.4.15)

In [2]:
safety_factor = 1.5


### Pattern size
For optimal strength and to maintain isotropic properties, the isogrid cells need to wrap around the tank diameter in integer increments. Twenty four cells was chosen as a starting point.

In [3]:
import math
num_radial_divisions = 12
pattern_size = grid_od*math.pi/num_radial_divisions
pattern_height = pattern_size*(3**(1/2))/2
print('Pattern Size: {:5.5f}'.format(pattern_size))
print('Pattern Height: {:5.5f}'.format(pattern_height))


Pattern Size: 3.12065
Pattern Height: 2.70256


### Design Properties

from "Isogrid Definitions", page viii

**Design choices:**<br>
t: skin thickness; &emsp;&emsp; minimum 0.02" per Peter<br> 
b: rib width; &emsp;&emsp;&emsp;&emsp;  min 0.03" per Peter<br>
d: rib depth; &emsp;&emsp;&emsp;&emsp; calculated from OD minus skin<br>
c: flange depth<br>
w: flange width<br>
s: plate thickness of unflanged isogrid<br>
h: triangle height<br>
a: triangle leg length (center to center along pattern)<br><br>

**Non-dimensional parameters:**<br>
δ: rib width to skin thickness ratio <br>
λ: flange depth to skin thickness ratio<br>
α: rib cross section to skin cross section ratio<br>
μ: flange cross section to skin cross section ratio<br>
β^2 = bending stiffness parameter<br><br>

**Equivalent material properties:**<br>
t_eff = equivalent thickness for membrane stresses<br>
t_bar = equivalent weight thickness <br>
t_star = equivalent monocoque thickness<br>
E_star = equivalent monocoque Young's modulus<br><br>

**Stiffness coefficients:**<br>
K: extensional stiffness<br>
D: bending stiffness<br>
I: moment of inertia


In [4]:
#Design choices
t = 0.02
b = 0.03
d = grid_wall_t-t
c = 0.0
w = 0.0
s = t + d
h = pattern_height
a = pattern_size

# non dimensional terms α λ μ δ β
δ = d/t
λ = c/t
α = b*d/t/h
μ = w*c/t/h
β_squared = (1+α+μ)*(3*(1+δ)**2+3*μ*(1+λ)**2+1+α*δ**2+μ*λ**2) - 3*(1+δ-μ*(1+λ))**2
β = β_squared**(1/2)

#Equivalent properties
t_eff = t*(1+α+μ)
t_bar = t*(1+3*(α+μ))
t_star = t*β/(1+α+μ)
E_star = E*(1+α+μ)**2/β

#stiffness parameters
K = E_star*t_star/(1-nu**2)
D = E_star*t_star**3/(12*(1-nu**2))
#I = (1+α+μ)*(3*(1+δ)**2+3*μ*(1+λ)**2+1+α*δ**2+μ*λ**2)-3*(1+δ-μ*(1+λ))**2
#I_beta = t_star**3*β_squared/(12*(1+α+μ))

## Tank Properties
The isogrid properties are calculated from NASA CR-124075

In [5]:
print('Web non-dimensional ratio, Alpha: {:5.3f}'.format(α))
print('Rib depth to skin thickness, Delta: {:5.3f}'.format(δ))
print('Bending stiffness parameter, Beta: {:5.3f}'.format(β))
print('Skin thickness, t: {:5.3f} in'.format(t))
print('Effective thickness, t_eff: {:5.3f} in'.format(t_eff))
print('Equivalant monocoque thickness, t*: {:5.3f} in'.format(t_star))
print('Equivalant weight thickness, t_bar: {:5.3f} in'.format(t_bar))
print('Equivalant monocoque Young\'s Modulus, E*: {:5.3f} psi'.format(E_star))
#print('Equivalant moment of Inertia, I: {:5.3f} in^4'.format(I))
#print('Equivalant beta sq amoment of Inertia, I: {:5.10f} in^4'.format(I_beta))

Web non-dimensional ratio, Alpha: 0.105
Rib depth to skin thickness, Delta: 9.500
Bending stiffness parameter, Beta: 6.820
Skin thickness, t: 0.020 in
Effective thickness, t_eff: 0.022 in
Equivalant monocoque thickness, t*: 0.123 in
Equivalant weight thickness, t_bar: 0.026 in
Equivalant monocoque Young's Modulus, E*: 1791961.057 psi


## Mass Model
Equations for calculating the mass of the isotank based on diameter and length

tbar: equiv wt thickness<br>

In [6]:
import math

cross_section = math.pi*((grid_id/2+t_bar)**2-(grid_id/2)**2)
volume = cross_section*grid_length
grid_mass = volume*density
lbm_to_kg = 0.453592 #unit conversion
grid_mass_kg = grid_mass * lbm_to_kg

print('grid_mass:',grid_mass, 'lbm')
print('grid_mass_kg:',grid_mass_kg, 'kg')
#print('tank_wall_mass:',tank_wall_mass, 'kg')

grid_mass: 1.974404972480726 lbm
grid_mass_kg: 0.8955743002774774 kg


## Load Calculations



### Combined Compression and Bending
For the fuel tank, the cylinder is under compression and bending. Section 4.2 on page 4.2.001 (#97) covers this scenario. The example covers:
* General Instability (N_cr_1)
* Skin Buckling (N_cr_2)
* Rib Crippling (N_cr_3)

In [7]:
import math

# Inputs
F = 1573 # lbf ~7000 N
M = 265 # lbf*in = 30 Nm

# Instability
Y = 1-0.407*(1-math.exp((-1/16)*(grid_r/t_eff)**(1/2))) #buckling knockdown factor: https://www.hindawi.com/journals/ijae/2020/9851984/#B4

c0 = 0.612*Y
c1 = 10.2     # 4.2.006
c2 = 0.616    # 4.2.008

R_t_star = grid_r/t_star
L_R = grid_length/grid_r

# Classical Value
N_cl = (1/(3*(1-nu**2))**0.5)*E_star*t_star**2/grid_r # k/in

N_a = F/(2*math.pi*grid_r) # k/in
N_b = M/(math.pi*grid_r**2) # k/in
N_cr = N_a + N_b
Ncr_Nb_check = N_cr/4
N_cr2 = E_star*t_star**2*(3*(1-nu)**2)**(-1/2)/grid_r

timoshenko = 0.85
N_cl_timo = N_cl*timoshenko

N_cr_1 = c0*E_star*t_star**2/grid_r*β
N_cr_2 = c1*E_star*t_star*(1+α)*(t_star**2)/(h**2)
N_cr_3 = c2*E_star*t_star*(1+α)*(b**2)/(d**2)

#p_burst = S_u*1000*skin_t*(1+alpha)/tank_r
#p_meop = p_burst/1.5

print('Na - ',N_a,'lbf/in')
print('Nb - ',N_b,'lbf/in')
print('Ncr - ',N_cr,'lbf/in')
print('Ncr_Nb_check ',Ncr_Nb_check,'\n')


print('If Ncr_Nb_check > Nb: \n edit timoshenko modifier using ref 2.1 (4.2.002) \n https://sci-hub.se/https://doi.org/10.1017/S0001924000014755')
print('\nTimoshenko lookup values\n R/t*',R_t_star)
print(' L/R - ',L_R)

print('\nBuckling - Classical',N_cl, 'lbf')
print('Buckling - Timoshenko',N_cl_timo, 'lbf')
print('\nGeneral Instability - ',N_cr_1,'k/in',N_cr_1*2*math.pi*grid_r,'lbf')
print('Skin Buckling       - ',N_cr_2,'k/in',N_cr_2*2*math.pi*grid_r,'lbf')
print('Rib Crippling        - ',N_cr_3,'k/in',N_cr_3*2*math.pi*grid_r,'lbf')
#print('P_burst             - ',p_burst,'psi')
#print('P_MEOP              - ',p_meop,'psi')


Na -  42.00515528247506 lbf/in
Nb -  2.3746711814418417 lbf/in
Ncr -  44.3798264639169 lbf/in
Ncr_Nb_check  11.094956615979225 

If Ncr_Nb_check > Nb: 
 edit timoshenko modifier using ref 2.1 (4.2.002) 
 https://sci-hub.se/https://doi.org/10.1017/S0001924000014755

Timoshenko lookup values
 R/t* 48.306275733920046
 L/R -  3.564

Buckling - Classical 2799.264044626936 lbf
Buckling - Timoshenko 2379.3744379328955 lbf

General Instability -  14113.518349224589 k/in 528519.9927017662 lbf
Skin Buckling       -  5195.7362645036255 k/in 194568.81159237158 lbf
Rib Crippling        -  3753.43211980674 k/in 140557.7168981272 lbf


## Stress Calcs

In [8]:
Fx = 0.0 #axial
Fy = F #circumferential
Txy = 0.0

Na = F/(2*math.pi*grid_r)
Nb = M/(math.pi*grid_r**2)
N_max = Na+N_b
max_stress_fiber = N_max/t_eff

#add in membrane rib and skin stresses (2.0.021)

print('axial load per inch: ',Na,'lbf/in')
print('max bending load per inch: ',Nb,'lbf/in')
print('combined: ',N_max,'lbf/in')
print('max stress fiber:',max_stress_fiber,'psi')

axial load per inch:  42.00515528247506 lbf/in
max bending load per inch:  2.3746711814418417 lbf/in
combined:  44.3798264639169 lbf/in
max stress fiber: 2007.3094475945504 psi


## Thermal Expansion (Contraction)

In [9]:
cte = 13*10**-6 # in/in F
dt = -297-70 # Room temperature to liquid oxygen temp
delta_diameter = cte*grid_od*dt
print(delta_diameter)

-0.056870319999999995
